In [16]:
 pip install midiutil

  Created wheel for midiutil: filename=MIDIUtil-1.2.1-py3-none-any.whl size=55492 sha256=e1d7b65fd5e0c5803253ff12281229afab3fac0dad84b834b5c68d758247420d
  Stored in directory: c:\users\camer\appdata\local\pip\cache\wheels\52\c2\30\ce4f23db030a863977bc328651b914261581717167aabfcd47
Successfully built midiutil
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install ephem

Note: you may need to restart the kernel to use updated packages.


In [3]:
import ephem
import numpy as np
from midiutil.MidiFile3 import MIDIFile

In [78]:
class Planet:
    def __init__(self, name):
        self.name = name
    
    def get_distance_from_sun(self):

        #sun = ephem.Sun()
        #sun.compute()
        planet = getattr(ephem, self.name)()
        planet.compute()
        distance_au = planet.sun_distance
        return distance_au
        
class Earth(Planet):
    def __init__(self, name):
        self.name = name
    
    def get_distance_from_sun(self):
        observer = ephem.Observer()
        observer.date = ephem.now()
        observer.lat = '0'
        observer.lon = '0'
        
        sun = ephem.Sun()
        sun.compute(observer)
        
        distance_au = ephem.separation(sun, observer)
        
        return distance_au

In [79]:
Mercury = Planet("Mercury")
Venus = Planet("Venus")
Earth = Earth("Earth")
Mars = Planet("Mars")
Jupiter = Planet("Jupiter")
Saturn = Planet("Saturn")
Uranus = Planet("Uranus")
Neptune = Planet("Neptune")

Planets = [Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune]
distances = []
distances_dict = {}

In [82]:
for planet in Planets:
    distance_au = planet.get_distance_from_sun()
    distances.append(distance_au)
    distances_dict[planet.name] = distance_au
    print(f"The distance between the Sun and {planet.name} is {distance_au:.2f} AU.")

The distance between the Sun and Mercury is 0.32 AU.
The distance between the Sun and Venus is 0.72 AU.
The distance between the Sun and Earth is 0.29 AU.
The distance between the Sun and Mars is 1.65 AU.
The distance between the Sun and Jupiter is 4.95 AU.
The distance between the Sun and Saturn is 9.81 AU.
The distance between the Sun and Uranus is 19.66 AU.
The distance between the Sun and Neptune is 29.91 AU.


In [92]:
T = 1000 # Tension
mu = 0.001 # Linear density

def frequency(distance, n=1):
    return n * np.power(T / mu, 1/2) / (2 * distance)

def freq_to_note(freq):
    notes = ['A', 'A#', 'B', 'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#']

    note_number = 12 * np.log2(freq / 440) + 49  
    note_number = round(note_number)
        
    note = (note_number - 1 ) % len(notes)
    note = notes[note]
    
    octave = (note_number + 8 ) // len(notes)
    
    return note, octave

def midi_pitch_number(freq):
    m  =  12 * np.log2(freq/440) + 69
    return m

In [93]:
notes = []
for planet in distances_dict:
    freq = frequency(distances_dict[planet])
    midi = midi_pitch_number(freq)
    note, octave = freq_to_note(freq)
    notes.append((freq, note, octave, midi))
    
notes

[(1574.0811090275208, 'G', 6, 91.06721343459654),
 (695.7260239070894, 'F', 5, 76.93218915051658),
 (1738.4405165996359, 'A', 6, 92.7866193003692),
 (302.7685797212594, 'D#', 4, 62.5285436571277),
 (100.95512901053333, 'G#', 2, 43.514528216298224),
 (50.96256620036261, 'G#', 1, 31.68007568565951),
 (25.438186799457398, 'G#', 0, 19.650770666905714),
 (16.716660499141636, 'C', 0, 12.38226059045114)]

In [94]:
# create your MIDI object
mf = MIDIFile(1)     # only 1 track
track = 0   # the only track

time = 0    # start at the beginning
mf.addTrackName(track, time, "Sample Track")
mf.addTempo(track, time, 120)

# add some notes
channel = 0
volume = 100

for _, _, _, midi in notes:
    
    pitch = round(midi) 
    time = 0 
    duration = 5 
    mf.addNote(track, channel, pitch, time, duration, volume)

with open("chord of right now.mid", 'wb') as outf:
    mf.writeFile(outf)